# Data Preprocessing

In [142]:
# load library
import numpy as np
import pandas as pd
import zipfile
import urllib.request
import io
import os
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GroupKFold

In [143]:
# load raw data
url = "https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/ehr_preprocessed_seq_by_day_cat_embedding.pkl.zip"
with urllib.request.urlopen(url) as response:
    with zipfile.ZipFile(io.BytesIO(response.read())) as zip_file:
        with zip_file.open("ehr_preprocessed_seq_by_day_cat_embedding.pkl", "r") as file:
            EHR = pd.read_pickle(file)
train = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/train.csv")
valid = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/valid.csv")
test = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/test.csv")
print(train.shape)
print(test.shape)
print(valid.shape)
print(len(EHR['feature_cols']))

(55941, 13)
(17933, 12)
(13598, 13)
171


In [144]:
# drop image-related columns
train = train.drop(train.columns[[6,7,8,11]], axis=1)
valid = valid.drop(valid.columns[[6,7,8,11]], axis=1)
test = test.drop(test.columns[[6,7,8,11]], axis=1)
train.head(5)

,id,subject_id,hadm_id,admittime,dischtime,deathtime,StudyDate,StudyTime,readmitted_within_30days
0,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410828,90151.343,0
1,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410831,130627.031,0
2,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410821,80228.937,0
3,17910612_22301530,17910612,22301530,2188-03-04 19:49:00,2188-04-19 00:00:00,2188-04-19 00:00:00,21880405,200636.062,1
4,17910612_22301530,17910612,22301530,2188-03-04 19:49:00,2188-04-19 00:00:00,2188-04-19 00:00:00,21880321,114135.046,1


In [145]:
# extract last-day observations
train['StudyDate'] = pd.to_datetime(train['StudyDate'], format='%Y%m%d')
train_latest_date_idxs = train.groupby('id')['StudyDate'].idxmax()
train_masked = train.loc[train_latest_date_idxs]

valid['StudyDate'] = pd.to_datetime(valid['StudyDate'], format='%Y%m%d')
valid_latest_date_idxs = valid.groupby('id')['StudyDate'].idxmax()
valid_masked = valid.loc[valid_latest_date_idxs]

test['StudyDate'] = pd.to_datetime(test['StudyDate'], format='%Y%m%d')
test_latest_date_idxs = test.groupby('id')['StudyDate'].idxmax()
test_masked = test.loc[test_latest_date_idxs]

print(train_masked.shape)
print(valid_masked.shape)
print(test_masked.shape)

(9271, 9)
(2325, 9)
(2936, 8)


In [146]:
# merge train and valid
train_valid = pd.concat([train, valid])
train_valid_masked = pd.concat([train_masked, valid_masked])
print(train_valid.shape)
print(train_valid_masked.shape)

(69539, 9)
(11596, 9)


In [147]:
# add two new predictors (length of stay + previous number of admissions)
# length of stay
train_masked['dischtime'] = pd.to_datetime(train_masked['dischtime'])
train_masked['admittime'] = pd.to_datetime(train_masked['admittime'])
train_masked['LoS'] = (train_masked['dischtime'] - train_masked['admittime']).dt.days
valid_masked['dischtime'] = pd.to_datetime(valid_masked['dischtime'])
valid_masked['admittime'] = pd.to_datetime(valid_masked['admittime'])
valid_masked['LoS'] = (valid_masked['dischtime'] - valid_masked['admittime']).dt.days
train_valid_masked['dischtime'] = pd.to_datetime(train_valid_masked['dischtime'])
train_valid_masked['admittime'] = pd.to_datetime(train_valid_masked['admittime'])
train_valid_masked['LoS'] = (train_valid_masked['dischtime'] - train_valid_masked['admittime']).dt.days

# previous number of admissions
train_masked = train_masked.sort_values('admittime')
train_masked['prev_admits'] = train_masked.groupby('subject_id').cumcount()
valid_masked = valid_masked.sort_values('admittime')
valid_masked['prev_admits'] = valid_masked.groupby('subject_id').cumcount()
train_valid_masked = train_valid_masked.sort_values('admittime')
train_valid_masked['prev_admits'] = train_valid_masked.groupby('subject_id').cumcount()

train_masked.head(5)

,id,subject_id,hadm_id,admittime,dischtime,deathtime,StudyDate,StudyTime,readmitted_within_30days,LoS,prev_admits
41856,17195991_23542772,17195991,23542772,2110-01-11 22:47:00,2110-01-18 10:25:00,NaN,2110-01-16,90654.546,0,6,0
24332,13721591_20342223,13721591,20342223,2110-02-09 18:13:00,2110-02-22 20:51:00,NaN,2110-02-19,41948.468,0,13,0
20119,19170541_22178312,19170541,22178312,2110-02-28 21:48:00,2110-03-12 17:47:00,NaN,2110-03-11,81842.812,0,11,0
51567,15554295_27705504,15554295,27705504,2110-03-09 03:54:00,2110-05-18 11:34:00,NaN,2110-05-04,60653.312,0,70,0
29152,17643026_29919541,17643026,29919541,2110-03-25 11:15:00,2110-03-29 17:17:00,NaN,2110-03-28,140521.453,0,4,0


In [148]:
# construct merged dataframes (admit info + features)
# train_valid set
train_valid_merged_admits_all = []
train_valid_merged_admits_latest = []
train_valid_merged_admits_median = []
for _, admit in train_valid_masked.iterrows():
    # sequence data
    num_feat_arrs = len(EHR['feat_dict'][admit['id']])
    for i in range(num_feat_arrs):
        feat_arr = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
        merged = pd.concat([admit, feat_arr])
        merged['obs_id'] = i + 1  
        train_valid_merged_admits_all.append(merged)
    # selected data
    feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][-1], index=EHR['feature_cols'])
    feat_arr_median = round(pd.Series(np.nanmedian(EHR['feat_dict'][admit['id']], axis=0), index=EHR['feature_cols']))
    train_valid_merged_admits_latest.append(pd.concat([admit, feat_arr_latest]))
    train_valid_merged_admits_median.append(pd.concat([admit, feat_arr_median]))
train_valid_merged_df_all = pd.DataFrame(train_valid_merged_admits_all)
train_valid_merged_df_latest = pd.DataFrame(train_valid_merged_admits_latest)
train_valid_merged_df_median = pd.DataFrame(train_valid_merged_admits_median)

# train set
train_merged_admits_all = []
train_merged_admits_latest = []
train_merged_admits_median = []
for _, admit in train_masked.iterrows():
    # sequence data
    num_feat_arrs = len(EHR['feat_dict'][admit['id']])
    for i in range(num_feat_arrs):
        feat_arr = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
        merged = pd.concat([admit, feat_arr])
        merged['obs_id'] = i + 1  
        train_merged_admits_all.append(merged)
    # selected data
    feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][-1], index=EHR['feature_cols'])
    feat_arr_median = round(pd.Series(np.nanmedian(EHR['feat_dict'][admit['id']], axis=0), index=EHR['feature_cols']))
    train_merged_admits_latest.append(pd.concat([admit, feat_arr_latest]))
    train_merged_admits_median.append(pd.concat([admit, feat_arr_median]))
train_merged_df_all = pd.DataFrame(train_merged_admits_all)
train_merged_df_latest = pd.DataFrame(train_merged_admits_latest)
train_merged_df_median = pd.DataFrame(train_merged_admits_median)

# valid set
valid_merged_admits_all = []
valid_merged_admits_latest = []
valid_merged_admits_median = []
for _, admit in valid_masked.iterrows():
    # sequence data
    num_feat_arrs = len(EHR['feat_dict'][admit['id']])
    for i in range(num_feat_arrs):
        feat_arr = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
        merged = pd.concat([admit, feat_arr])
        merged['obs_id'] = i + 1  
        valid_merged_admits_all.append(merged)
    # selected data
    feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][-1], index=EHR['feature_cols'])
    feat_arr_median = round(pd.Series(np.nanmedian(EHR['feat_dict'][admit['id']], axis=0), index=EHR['feature_cols']))
    valid_merged_admits_latest.append(pd.concat([admit, feat_arr_latest]))
    valid_merged_admits_median.append(pd.concat([admit, feat_arr_median]))
valid_merged_df_all = pd.DataFrame(valid_merged_admits_all)
valid_merged_df_latest = pd.DataFrame(valid_merged_admits_latest)
valid_merged_df_median = pd.DataFrame(valid_merged_admits_median)

In [149]:
# standardization for numeric features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

cat_cols = [EHR['feature_cols'][i] for i in EHR['cat_idxs']]
icd_cols = EHR['icd_cols']
num_cols = list(set(EHR['feature_cols']) - set(cat_cols) - set(icd_cols)) 
num_cols.extend(['LoS', 'prev_admits']) 

train_valid_merged_df_all[num_cols] = scaler.fit_transform(train_valid_merged_df_all[num_cols])
train_valid_merged_df_median[num_cols] = scaler.fit_transform(train_valid_merged_df_median[num_cols])
train_valid_merged_df_latest[num_cols] = scaler.fit_transform(train_valid_merged_df_latest[num_cols])
train_merged_df_all[num_cols] = scaler.fit_transform(train_merged_df_all[num_cols])
train_merged_df_median[num_cols] = scaler.fit_transform(train_merged_df_median[num_cols])
train_merged_df_latest[num_cols] = scaler.fit_transform(train_merged_df_latest[num_cols])
valid_merged_df_all[num_cols] = scaler.fit_transform(valid_merged_df_all[num_cols])
valid_merged_df_median[num_cols] = scaler.fit_transform(valid_merged_df_median[num_cols])
valid_merged_df_latest[num_cols] = scaler.fit_transform(valid_merged_df_latest[num_cols])

In [150]:
# drop features with constant values in the training set
train_valid_merged_df_all.insert(2, 'obs_id', train_valid_merged_df_all.pop('obs_id'))
train_merged_df_all.insert(2, 'obs_id', train_merged_df_all.pop('obs_id'))
valid_merged_df_all.insert(2, 'obs_id', valid_merged_df_all.pop('obs_id'))
feat_cols_all = train_merged_df_all.columns[10:].to_list()
feat_cols_latest = train_merged_df_latest.columns[9:].to_list()
feat_cols_median = train_merged_df_median.columns[9:].to_list()
const_cols_all = [col for col in feat_cols_all if train_merged_df_all[col].nunique() <= 1]
const_cols_latest = [col for col in feat_cols_latest if train_merged_df_latest[col].nunique() <= 1]
const_cols_median = [col for col in feat_cols_median if train_merged_df_median[col].nunique() <= 1]
print(const_cols_all)
print(const_cols_latest)
print(const_cols_median)

train_valid_merged_df_all = train_valid_merged_df_all.drop(columns=const_cols_all, axis=1)
train_merged_df_all = train_merged_df_all.drop(columns=const_cols_all, axis=1)
valid_merged_df_all = valid_merged_df_all.drop(columns=const_cols_all, axis=1)
train_valid_merged_df_latest = train_valid_merged_df_latest.drop(columns=const_cols_latest, axis=1)
train_merged_df_latest = train_merged_df_latest.drop(columns=const_cols_latest, axis=1)
valid_merged_df_latest = valid_merged_df_latest.drop(columns=const_cols_latest, axis=1)
train_valid_merged_df_median = train_valid_merged_df_median.drop(columns=const_cols_median, axis=1)
train_merged_df_median = train_merged_df_median.drop(columns=const_cols_median, axis=1)
valid_merged_df_median = valid_merged_df_median.drop(columns=const_cols_median, axis=1)

print(train_valid_merged_df_all.shape)
print(train_valid_merged_df_latest.shape)
print(train_valid_merged_df_median.shape)

['H30-H36', 'N00-N08', 'C73-C75', 'M60-M63', 'A20-A28', 'M50-M54', 'N25-N29', 'J00-J06', 'M05-M14', 'K70-K77', 'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS']
['H30-H36', 'N00-N08', 'C73-C75', 'M60-M63', 'A20-A28', 'M50-M54', 'N25-N29', 'J00-J06', 'M05-M14', 'K70-K77', 'Basophils Other Body Fluid', 'Basophils Pleural', 'Eosinophils Joint Fluid', 'pH Urine', 'Basophils Joint Fluid', 'Basophils Ascites', 'Lymphocytes Joint Fluid', 'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS']
['H30-H36', 'N00-N08', 'C73-C75', 'M60-M63', 'A20-A28', 'M50-M54', 'N25-N29', 'J00-J06', 'M05-M14', 'K70-K77', 'Basophils Other Body Fluid', 'Basophils Pleural', 'Eosinophils Joint Fluid', 'Eosinophils Ascites', 'pH Urine', 'Lymphocytes Ascites', 'Eosinophils Other Body Fluid', 'Basophils Joint Fluid', 'H Blood', 'Basophils Ascites', 'Lymphocytes Other Body Fluid', 'Monocytes Ascites', 'Lymphocytes Joint Fluid', 'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS']
(173154, 172)
(11596, 164)
(11596, 15

In [151]:
# make copies
train_valid_merged_df_all_copy = train_valid_merged_df_all.copy()
train_merged_df_all_copy = train_merged_df_all.copy()
train_valid_merged_df_latest_copy = train_valid_merged_df_latest.copy()
train_merged_df_latest_copy = train_merged_df_latest.copy()
train_valid_merged_df_median_copy = train_valid_merged_df_median.copy()
train_merged_df_median_copy = train_merged_df_median.copy()

In [152]:
# drop info columns
train_valid_merged_df_all = train_valid_merged_df_all.drop(train_valid_merged_df_all.columns[2:9], axis=1)
train_merged_df_all = train_merged_df_all.drop(train_merged_df_all.columns[2:9], axis=1)
train_valid_merged_df_latest = train_valid_merged_df_latest.drop(train_valid_merged_df_latest.columns[2:8], axis=1)
train_merged_df_latest = train_merged_df_latest.drop(train_merged_df_latest.columns[2:8], axis=1)
train_valid_merged_df_median = train_valid_merged_df_median.drop(train_valid_merged_df_median.columns[2:8], axis=1)
train_merged_df_median = train_merged_df_median.drop(train_merged_df_median.columns[2:8], axis=1)

In [153]:
# feature selection
# 1. drop features with correlation (with response) lower than 0.1 (5/10 fold correlation test)
# 2. if multiple features have correlation > 0.7 between each other, only keep the one with the highest correlation with response

#train_merged_df_all
#STEP1
k = 10
cv = GroupKFold(n_splits=k)
selected_features = []
for train_index, _ in cv.split(train_merged_df_all, groups=train_merged_df_all['subject_id']):
    fold_data = train_merged_df_all.iloc[train_index]
    numeric_data = fold_data.select_dtypes(include=np.number) 
    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
    selected_features.append(correlations)
#compute average correlation
selected_features_df = pd.concat(selected_features, axis=1)
average_correlation = selected_features_df.mean(axis=1)
#drop variables with correlation with response lower than 0.01
selected_columns = average_correlation[average_correlation < 0.01].index
train_selected_df_all = train_merged_df_all.drop(selected_columns, axis=1)
#STEP2
corr_matrix = train_selected_df_all.corr()
cols = corr_matrix.columns
#drop variables with correlation with correlation between each other>0.7 except the highest one
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if abs(corr_matrix.iloc[i,j]) > 0.7:
            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
                train_selected_df_all = train_selected_df_all.drop(cols[j], axis=1)
            else:
                train_selected_df_all = train_selected_df_all.drop(cols[i], axis=1)
                break
train_selected_df_all = train_selected_df_all.drop(columns=['id'])
print(train_selected_df_all.shape)

#train_merged_df_latest
#STEP1
k = 10
cv = GroupKFold(n_splits=k)
selected_features = []
for train_index, _ in cv.split(train_merged_df_latest, groups=train_merged_df_latest['subject_id']):
    fold_data = train_merged_df_latest.iloc[train_index]
    numeric_data = fold_data.select_dtypes(include=np.number) 
    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
    selected_features.append(correlations)
selected_features_df = pd.concat(selected_features, axis=1)
average_correlation = selected_features_df.mean(axis=1)
selected_columns = average_correlation[average_correlation < 0.01].index
train_selected_df_latest = train_merged_df_latest.drop(selected_columns, axis=1)
#STEP2
corr_matrix = train_selected_df_latest.corr()
cols = corr_matrix.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if abs(corr_matrix.iloc[i,j]) > 0.7:
            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
                train_selected_df_latest = train_selected_df_latest.drop(cols[j], axis=1)
            else:
                train_selected_df_latest = train_selected_df_latest.drop(cols[i], axis=1)
                break
train_selected_df_latest = train_selected_df_latest.drop(columns=['id'])
print(train_selected_df_latest.shape)

#train_merged_df_median
#STEP1
k = 10
cv = GroupKFold(n_splits=k)
selected_features = []
for train_index, _ in cv.split(train_merged_df_median, groups=train_merged_df_median['subject_id']):
    fold_data = train_merged_df_median.iloc[train_index]
    numeric_data = fold_data.select_dtypes(include=np.number) 
    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
    selected_features.append(correlations)
selected_features_df = pd.concat(selected_features, axis=1)
average_correlation = selected_features_df.mean(axis=1)
selected_columns = average_correlation[average_correlation < 0.01].index
train_selected_df_median = train_merged_df_median.drop(selected_columns, axis=1)
#STEP2
corr_matrix = train_selected_df_median.corr()
cols = corr_matrix.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if abs(corr_matrix.iloc[i,j]) > 0.7:
            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
                train_selected_df_median = train_selected_df_median.drop(cols[j], axis=1)
            else:
                train_selected_df_median = train_selected_df_median.drop(cols[i], axis=1)
                break
train_selected_df_median = train_selected_df_median.drop(columns=['id'])
print(train_selected_df_median.shape)

/var/folders/sy/913ynw2s0pb10kx9y5qmlwnm0000gn/T/ipykernel_9505/197208685.py:22: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train_selected_df_all.corr()


(138851, 52)


/var/folders/sy/913ynw2s0pb10kx9y5qmlwnm0000gn/T/ipykernel_9505/197208685.py:51: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train_selected_df_latest.corr()


(9271, 54)
(9271, 51)


/var/folders/sy/913ynw2s0pb10kx9y5qmlwnm0000gn/T/ipykernel_9505/197208685.py:79: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train_selected_df_median.corr()


In [154]:
#train_valid_merged_df_all
#STEP1
k = 10
cv = GroupKFold(n_splits=k)
selected_features = []
for train_valid_index, _ in cv.split(train_valid_merged_df_all, groups=train_valid_merged_df_all['subject_id']):
    fold_data = train_valid_merged_df_all.iloc[train_valid_index]
    numeric_data = fold_data.select_dtypes(include=np.number) 
    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
    selected_features.append(correlations)
#compute average correlation
selected_features_df = pd.concat(selected_features, axis=1)
average_correlation = selected_features_df.mean(axis=1)
#drop variables with correlation with response lower than 0.01
selected_columns = average_correlation[average_correlation < 0.01].index
train_valid_selected_df_all = train_valid_merged_df_all.drop(selected_columns, axis=1)
#STEP2
corr_matrix = train_valid_selected_df_all.corr()
cols = corr_matrix.columns
#drop variables with correlation with correlation between each other>0.7 except the highest one
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if abs(corr_matrix.iloc[i,j]) > 0.7:
            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
                train_valid_selected_df_all = train_valid_selected_df_all.drop(cols[j], axis=1)
            else:
                train_valid_selected_df_all = train_valid_selected_df_all.drop(cols[i], axis=1)
                break
train_valid_selected_df_all = train_valid_selected_df_all.drop(columns=['id'])
print(train_valid_selected_df_all.shape)

#train_valid_merged_df_latest
#STEP1
k = 10
cv = GroupKFold(n_splits=k)
selected_features = []
for train_valid_index, _ in cv.split(train_valid_merged_df_latest, groups=train_valid_merged_df_latest['subject_id']):
    fold_data = train_valid_merged_df_latest.iloc[train_valid_index]
    numeric_data = fold_data.select_dtypes(include=np.number) 
    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
    selected_features.append(correlations)
selected_features_df = pd.concat(selected_features, axis=1)
average_correlation = selected_features_df.mean(axis=1)
selected_columns = average_correlation[average_correlation < 0.01].index
train_valid_selected_df_latest = train_valid_merged_df_latest.drop(selected_columns, axis=1)
#STEP2
corr_matrix = train_valid_selected_df_latest.corr()
cols = corr_matrix.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if abs(corr_matrix.iloc[i,j]) > 0.7:
            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
                train_valid_selected_df_latest = train_valid_selected_df_latest.drop(cols[j], axis=1)
            else:
                train_valid_selected_df_latest = train_valid_selected_df_latest.drop(cols[i], axis=1)
                break
train_valid_selected_df_latest = train_valid_selected_df_latest.drop(columns=['id'])
print(train_valid_selected_df_latest.shape)

#train_valid_merged_df_median
#STEP1
k = 10
cv = GroupKFold(n_splits=k)
selected_features = []
for train_valid_index, _ in cv.split(train_valid_merged_df_median, groups=train_valid_merged_df_median['subject_id']):
    fold_data = train_valid_merged_df_median.iloc[train_valid_index]
    numeric_data = fold_data.select_dtypes(include=np.number) 
    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
    selected_features.append(correlations)
selected_features_df = pd.concat(selected_features, axis=1)
average_correlation = selected_features_df.mean(axis=1)
selected_columns = average_correlation[average_correlation < 0.01].index
train_valid_selected_df_median = train_valid_merged_df_median.drop(selected_columns, axis=1)
#STEP2
corr_matrix = train_valid_selected_df_median.corr()
cols = corr_matrix.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if abs(corr_matrix.iloc[i,j]) > 0.7:
            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
                train_valid_selected_df_median = train_valid_selected_df_median.drop(cols[j], axis=1)
            else:
                train_valid_selected_df_median = train_valid_selected_df_median.drop(cols[i], axis=1)
                break
train_valid_selected_df_median = train_valid_selected_df_median.drop(columns=['id'])
print(train_valid_selected_df_median.shape)

/var/folders/sy/913ynw2s0pb10kx9y5qmlwnm0000gn/T/ipykernel_9505/114909104.py:18: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train_valid_selected_df_all.corr()


(173154, 49)


/var/folders/sy/913ynw2s0pb10kx9y5qmlwnm0000gn/T/ipykernel_9505/114909104.py:47: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train_valid_selected_df_latest.corr()


(11596, 53)
(11596, 50)


/var/folders/sy/913ynw2s0pb10kx9y5qmlwnm0000gn/T/ipykernel_9505/114909104.py:75: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train_valid_selected_df_median.corr()


In [156]:
# write csv files
# train set
id_cols = train_merged_df_all_copy[['obs_id', 'admittime', 'dischtime']]
train_merged_df_all = pd.concat([id_cols, train_merged_df_all], axis=1)
id_cols = train_merged_df_latest_copy[['admittime', 'dischtime']]
train_merged_df_latest = pd.concat([id_cols, train_merged_df_latest], axis=1)
id_cols = train_merged_df_median_copy[['admittime', 'dischtime']]
train_merged_df_median = pd.concat([id_cols, train_merged_df_median], axis=1)

id_cols = train_valid_merged_df_all_copy[['obs_id', 'admittime', 'dischtime']]
train_valid_merged_df_all = pd.concat([id_cols, train_valid_merged_df_all], axis=1)
id_cols = train_valid_merged_df_latest_copy[['admittime', 'dischtime']]
train_valid_merged_df_latest = pd.concat([id_cols, train_valid_merged_df_latest], axis=1)
id_cols = train_valid_merged_df_median_copy[['admittime', 'dischtime']]
train_valid_merged_df_median = pd.concat([id_cols, train_valid_merged_df_median], axis=1)

id_cols = train_merged_df_all_copy[['id','subject_id', 'obs_id', 'admittime', 'dischtime']]
train_selected_df_all = pd.concat([id_cols, train_selected_df_all], axis=1)
id_cols = train_merged_df_latest_copy[['id','subject_id', 'admittime', 'dischtime']]
train_selected_df_latest = pd.concat([id_cols, train_selected_df_latest], axis=1)
id_cols = train_merged_df_median_copy[['id','subject_id', 'admittime', 'dischtime']]
train_selected_df_median = pd.concat([id_cols, train_selected_df_median], axis=1)

id_cols = train_valid_merged_df_all_copy[['id','subject_id', 'obs_id', 'admittime', 'dischtime']]
train_valid_selected_df_all = pd.concat([id_cols, train_valid_selected_df_all], axis=1)
id_cols = train_valid_merged_df_latest_copy[['id','subject_id', 'admittime', 'dischtime']]
train_valid_selected_df_latest = pd.concat([id_cols, train_valid_selected_df_latest], axis=1)
id_cols = train_valid_merged_df_median_copy[['id','subject_id', 'admittime', 'dischtime']]
train_valid_selected_df_median = pd.concat([id_cols, train_valid_selected_df_median], axis=1)

train_merged_df_latest.to_csv('train/all_feat/train_latest.csv', index=False)
train_selected_df_latest.to_csv('train/selected_feat/train_latest.csv', index=False)
train_valid_merged_df_latest.to_csv('train/all_feat/train_valid_latest.csv', index=False)
train_valid_selected_df_latest.to_csv('train/selected_feat/train_valid_latest.csv', index=False)

train_merged_df_median.to_csv('train/all_feat//train_median.csv', index=False)
train_selected_df_median.to_csv('train/selected_feat/train_median.csv', index=False)
train_valid_merged_df_median.to_csv('train/all_feat/train_valid_median.csv', index=False)
train_valid_selected_df_median.to_csv('train/selected_feat/train_valid_median.csv', index=False)

train_merged_df_all.to_csv('train/all_feat//train_all.csv', index=False)
train_valid_merged_df_all.to_csv('train/all_feat/train_valid_all.csv', index=False)
train_selected_df_all.to_csv('train/selected_feat/train_all.csv', index=False)
train_valid_selected_df_all.to_csv('train/selected_feat/train_valid_all.csv', index=False)
with zipfile.ZipFile('train/all_feat/sequence_data.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('train/all_feat/train_all.csv')
    zipf.write('train/all_feat/train_valid_all.csv')
with zipfile.ZipFile('train/selected_feat/sequence_data.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('train/selected_feat/train_all.csv')
    zipf.write('train/selected_feat/train_valid_all.csv')
os.remove('train/all_feat/train_all.csv')
os.remove('train/selected_feat/train_all.csv')
os.remove('train/all_feat/train_valid_all.csv')
os.remove('train/selected_feat/train_valid_all.csv')

In [158]:
# write csv files
# valid set
valid_merged_df_all = valid_merged_df_all[train_merged_df_all.columns]
valid_merged_df_all = valid_merged_df_all.reindex(columns=['id','subject_id','obs_id','admittime','dischtime'] + list(valid_merged_df_all.columns[5:]))
valid_seleted_df_all = valid_merged_df_all[train_selected_df_all.columns]
valid_merged_df_latest = valid_merged_df_latest[train_merged_df_latest.columns]
valid_merged_df_latest = valid_merged_df_latest.reindex(columns=['id','subject_id','admittime','dischtime'] + list(valid_merged_df_latest.columns[4:]))
valid_seleted_df_latest = valid_merged_df_latest[train_selected_df_latest.columns] 
valid_merged_df_median = valid_merged_df_median[train_merged_df_median.columns]
valid_merged_df_median = valid_merged_df_median.reindex(columns=['id','subject_id','admittime','dischtime'] + list(valid_merged_df_median.columns[4:]))
valid_seleted_df_median = valid_merged_df_median[train_selected_df_median.columns]

valid_merged_df_all.to_csv('valid/all_feat/valid_all.csv', index=False)
valid_merged_df_latest.to_csv('valid/all_feat/valid_latest.csv', index=False)
valid_merged_df_median.to_csv('valid/all_feat/valid_median.csv', index=False)
valid_seleted_df_all.to_csv('valid/selected_feat/valid_all.csv', index=False)
valid_seleted_df_latest.to_csv('valid/selected_feat/valid_latest.csv', index=False)
valid_seleted_df_median.to_csv('valid/selected_feat/valid_median.csv', index=False)